<a href="https://colab.research.google.com/github/lpoggetto/fiap_liveness/blob/main/trabalho_computer_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!mkdir imagens
!rm -rf sample_data

mkdir: cannot create directory ‘imagens’: File exists


In [12]:
 # instalando pacotes utilizados

 !pip install git+https://github.com/hukkelas/DSFD-Pytorch-Inference.git
 !pip install face_detection
 !pip install mediapipe
 !pip install dlib

  Cloning https://github.com/hukkelas/DSFD-Pytorch-Inference.git to /tmp/pip-req-build-uxowmn5n
  Running command git clone --filter=blob:none --quiet https://github.com/hukkelas/DSFD-Pytorch-Inference.git /tmp/pip-req-build-uxowmn5n
  Resolved https://github.com/hukkelas/DSFD-Pytorch-Inference.git to commit dde9c7dd9cdc9254c2ca345222c86a8ecfa17f5b
  Preparing metadata (setup.py) ... done


### Importando os pacotes utilizados

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import bz2
import os
try:
  import face_detection
except:
  !pip install face_detection
  import face_detection

try:
  import mediapipe as mp
except:
  import mediapipe as mp
from scipy.spatial import distance as dist
import dlib
import warnings
warnings.filterwarnings('ignore')

#Exibição na mesma tela do Jupyter
%matplotlib inline
sns.set_style("whitegrid", {'axes.grid' : False})

# pacotes utilizados para tirar foto
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode

from flask import Flask, request, send_from_directory
import threading

In [19]:
# URL of the pre-trained model
url = "http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"
file_name = "shape_predictor_68_face_landmarks.dat.bz2"

# Download the file if it doesn't exist
if not os.path.exists(file_name):
  !wget {url}

# Extract the .dat file if it doesn't exist
dat_file = "shape_predictor_68_face_landmarks.dat"
if not os.path.exists(dat_file):
  with bz2.BZ2File(file_name, "rb") as f_in, open(dat_file, "wb") as f_out:
    f_out.write(f_in.read())

In [ ]:
# funcao para tirar foto
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

try:
  filename = take_photo("imagens/foto.jpg")
  print('Saved to {}'.format(filename))

  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

<IPython.core.display.Javascript object>

Saved to imagens/foto.jpg
'module' object is not callable


In [ ]:
from IPython.display import HTML, display

# JavaScript code to capture video
html_code = """
<video id="video" width="640" height="480" autoplay></video>
<button id="startButton">Start Recording</button>
<button id="stopButton" disabled>Stop Recording</button>
<video id="playback" width="640" height="480" controls></video>
<a id="downloadLink" download="recorded-video.webm"></a>

<script>
    const video = document.querySelector('#video');
    const playback = document.querySelector('#playback');
    const downloadLink = document.querySelector('#downloadLink');
    const startButton = document.querySelector('#startButton');
    const stopButton = document.querySelector('#stopButton');

    navigator.mediaDevices.getUserMedia({ video: true }).then((stream) => {
        video.srcObject = stream;
        let mediaRecorder;
        const chunks = [];

        startButton.onclick = () => {
            mediaRecorder = new MediaRecorder(stream);
            mediaRecorder.start();
            startButton.disabled = true;
            stopButton.disabled = false;

            mediaRecorder.ondataavailable = (event) => chunks.push(event.data);
            mediaRecorder.onstop = () => {
                const blob = new Blob(chunks, { type: 'video/webm' });
                playback.src = URL.createObjectURL(blob);
                downloadLink.href = playback.src;
                downloadLink.textContent = 'Download Video';
                startButton.disabled = false;
                stopButton.disabled = true;
            };
        };

        stopButton.onclick = () => mediaRecorder.stop();
    });
</script>
"""

# Display the HTML interface
display(HTML(html_code))

In [ ]:
from google.colab import files
import os
from IPython.display import HTML, display

# Ensure the output folder exists
output_folder = "/content/videos/"
os.makedirs(output_folder, exist_ok=True)

# HTML + JavaScript for video recording and upload
html_code = """
<video id="video" width="640" height="480" autoplay></video>
<button id="startButton">Start Recording</button>
<button id="stopButton" disabled>Stop Recording</button>
<video id="playback" width="640" height="480" controls></video>
<a id="downloadLink" style="display: none;"></a>

<script>
    const video = document.querySelector('#video');
    const playback = document.querySelector('#playback');
    const downloadLink = document.querySelector('#downloadLink');
    const startButton = document.querySelector('#startButton');
    const stopButton = document.querySelector('#stopButton');

    navigator.mediaDevices.getUserMedia({ video: true }).then((stream) => {
        video.srcObject = stream;
        let mediaRecorder;
        const chunks = [];

        startButton.onclick = () => {
            mediaRecorder = new MediaRecorder(stream);
            mediaRecorder.start();
            startButton.disabled = true;
            stopButton.disabled = false;

            mediaRecorder.ondataavailable = (event) => chunks.push(event.data);
            mediaRecorder.onstop = () => {
                const blob = new Blob(chunks, { type: 'video/webm' });
                const formData = new FormData();
                formData.append("file", blob, "recorded-video.webm");

                // Send the video blob to Python backend
                fetch("/upload", {
                    method: "POST",
                    body: formData
                }).then(response => response.text())
                  .then(result => alert("Video uploaded successfully!"));

                playback.src = URL.createObjectURL(blob);
                startButton.disabled = false;
                stopButton.disabled = true;
            };
        };

        stopButton.onclick = () => mediaRecorder.stop();
    });
</script>
"""

# Backend endpoint to handle file uploads
app = Flask(__name__)

@app.route('/upload', methods=['POST'])
def upload():
    file = request.files['file']
    file_path = os.path.join(output_folder, file.filename)
    file.save(file_path)
    return "File uploaded successfully!"

# Start the Flask app in a separate thread
def start_server():
    app.run(port=8000, debug=False)

thread = threading.Thread(target=start_server)
thread.start()

# Display the HTML interface
display(HTML(html_code))

### Montando detector de faces

Utilizando o algoritmo DSFDDetector para detectar uma face

In [ ]:
def detector_faces(path_foto):
  # Criando o detector de faces
  detector = face_detection.build_detector(
        "DSFDDetector",
        confidence_threshold=.5,
        nms_iou_threshold=.3
        )

  # Convertendo BGR -> RGB
  im = cv2.imread(path_foto)[:, :, ::-1]

  # array com as faces detectadas
  detections = detector.detect(im)

  # iterando para desenhar retangulo na face
  for det in detections:
      x1, y1, x2, y2, score = det  # obtendo resultados
      x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])  # convertendo para int

      # array na formatacao
      im = im.astype(np.uint8)

      # desenhando o retangulo na face detectada
      cv2.rectangle(im, (x1, y1), (x2, y2), (255, 0, 0), 2)

# Plot the image with bounding boxes and lines
  plt.figure(figsize=(10, 8))
  plt.imshow(im)
  plt.axis("off")
  plt.show()

In [30]:
def calculo_distancia_olhos(eye):
    # Calcula a distância euclidiana entre os pontos dos olhos
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    # Calcula a razão de aspecto do olho
    ear = (A + B) / (2.0 * C)
    return ear

def detector_faces_video(video_path, output_path, threshhold_ear=0.3):
    # Criando o detector de faces
    detector = face_detection.build_detector(
        "DSFDDetector",
        confidence_threshold=0.5,
        nms_iou_threshold=0.3
    )
    # utilizando algoritmo de 68 pontos faciais
    preditor_marcos_faciais = dlib.shape_predictor(
        '/content/shape_predictor_68_face_landmarks.dat'
        )

    # abrindo o arquivo de video
    video = cv2.VideoCapture(video_path)

    # propriedades do video
    fps = int(video.get(cv2.CAP_PROP_FPS))
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'VP80')  # WebM codec
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while True:
        ret, frame = video.read()
        if not ret:
            break

        # convertendo para RGB
        rgb_frame = frame[:, :, ::-1]

        # detectando  face
        detections = detector.detect(rgb_frame)

        # desenhando o retangulo na face
        for det in detections:
            x1, y1, x2, y2, score = det
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

            # utilizando dlib para detectar a face
            dlib_rect = dlib.rectangle(x1,y1,x2,y2)

            # marcos faciais
            landmarks = preditor_marcos_faciais(rgb_frame, dlib_rect)

            # coordenadas dos olhos
            left_eye = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(36, 42)]
            right_eye = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(42, 48)]

            # calculando a distancia para ambos os olhos
            olho_esq = calculo_distancia_olhos(left_eye)
            olho_dir = calculo_distancia_olhos(right_eye)
            ear = (olho_esq + olho_dir) / 2

            # validacao de olhos piscando
            blink_detected = ear < threshhold_ear
            if ear < threshhold_ear:
              cv2.putText(frame, 'Cliente Piscou', (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
              cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            else:
              cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)

        # retorno do video
        out.write(frame)

    # Release resources
    video.release()
    out.release()
    print(f"Processed video saved to: {output_path}")

# Example usage
input_video_path = '/content/imagens/recorded-video.webm'
output_video_path = '/content/imagens/recorded-video_processado.webm'

# chamada da funcao criada anteriormente
detector_faces_video(input_video_path, output_video_path)

Processed video saved to: /content/imagens/recorded-video_processado.webm
